In [1]:
grid = []

chars = set()

with open("./data/day23.txt") as f:
    while line := f.readline():
        line = line.rstrip()
        grid.append(list(line))
        for c in line:
            chars.add(c)

rows, cols = len(grid), len(grid[0])
chars

{'#', '.', '>', 'v'}

In [2]:
def get_directions(row, col, path):
    if grid[row][col] == '>':
        return [(row, col+1)] if (row, col+1) not in path else []
    if grid[row][col] == 'v':
        return [(row+1, col)] if (row+1, col) not in path else []
    res = []
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if (row+i, col+j) not in path and row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res.append((row+i, col+j))
    return res

In [3]:
q = [(0, 1, set([(0, 1),]))]
res = 0
while q:
    row, col, path = q.pop()
    if row == rows-1:
        res = max(res, len(path))
        continue
    for new_row, new_col in get_directions(row, col, path):
        new_path = path.copy()
        new_path.add((new_row, new_col))
        q.append((new_row, new_col, new_path))
res-1

2406

In [4]:
def get_directions2(row, col, path):
    res = []
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if (row+i, col+j) not in path and row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res.append((row+i, col+j))
    return res

In [5]:
def is_fork(row, col):
    if grid[row][col] == '#':
        return False
    res = 0
    for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        if row+i in range(rows) and col+j in range(cols) and grid[row+i][col+j] != '#':
            res += 1
    return res > 2

In [6]:
forks = []
forks.append((0, 1))
forks.append((rows-1, cols-2))
for row in range(rows):
    for col in range(cols):
        if is_fork(row, col):
            forks.append((row, col))
len(forks)

36

In [7]:
from collections import deque

In [8]:
fork_graph = {fork:{} for fork in forks}
for r, c in forks:
    q = deque()
    q.append((r, c, set([(r, c),])))
    while q:
        row, col, path = q.pop()
        if (row, col) != (r, c) and (row, col) in forks:
            fork_graph[(r, c)][(row, col)] = len(path)-1
            continue
        ls = get_directions2(row, col, path)
        for new_row, new_col in ls:
            if len(ls) > 1:
                new_path = path.copy()
            else:
                new_path = path
            new_path.add((new_row, new_col))
            q.appendleft((new_row, new_col, new_path))
for i, (k, v) in enumerate(fork_graph.items()):
    if i >= 10:
        break
    print(f'Fork {k} immediate neighbors and longest paths to: {v}')

Fork (0, 1) immediate neighbors and longest paths to: {(13, 11): 155}
Fork (140, 139) immediate neighbors and longest paths to: {(129, 123): 135}
Fork (5, 113) immediate neighbors and longest paths to: {(37, 113): 208, (13, 83): 278, (43, 129): 434}
Fork (13, 11) immediate neighbors and longest paths to: {(33, 15): 116, (0, 1): 155, (19, 43): 290}
Fork (13, 83) immediate neighbors and longest paths to: {(35, 89): 120, (17, 59): 236, (5, 113): 278}
Fork (17, 59) immediate neighbors and longest paths to: {(29, 61): 58, (19, 43): 134, (13, 83): 236}
Fork (19, 43) immediate neighbors and longest paths to: {(33, 31): 82, (17, 59): 134, (13, 11): 290}
Fork (29, 61) immediate neighbors and longest paths to: {(17, 59): 58, (35, 89): 142, (33, 31): 146, (63, 53): 222}
Fork (33, 15) immediate neighbors and longest paths to: {(33, 31): 64, (13, 11): 116, (67, 13): 316}
Fork (33, 31) immediate neighbors and longest paths to: {(33, 15): 64, (19, 43): 82, (29, 61): 146, (65, 29): 218}


In [9]:
q = [(0, 1, set([(0, 1),]), 0)]
res = 0
while q:
    row, col, path, d = q.pop()
    if row == rows-1:
        res = max(res, d)
        continue
    adj = fork_graph[(row, col)]
    for (new_row, new_col), dist in adj.items():
        if (new_row, new_col) in path:
            continue
        if len(adj) > 1:
            new_path = path.copy()
        else:
            new_path = path
        new_path.add((new_row, new_col))
        q.append((new_row, new_col, new_path, d+dist))
res

6630